# Project detection math expression

## Imports

In [3]:
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, utils
from collections import Counter
from PIL import Image
from tqdm import tqdm

import os
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models.detection as detection
# Install pix2text robustly using the current Python executable
import sys, subprocess
try:
    import pix2text
except Exception:
    print('pix2text not found — installing via python -m pip')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pix2text>=1.1'])
    import pix2text
print('pix2text is available')
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()  

[WARNING 2025-12-04 10:43:49,264 _showwarnmsg:109] /home/raclax/Documents/M2/Part2/DL2/Project/deepl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 


pix2text is available


## Pre treatement

In [ ]:
class CROHMEDataset(Dataset):
    """
    Dataset pour les expressions complètes (PNG + LG).
    Chaque sample retourne :
        - image : Tensor CxHxW
        - target : dict contenant "boxes" et "labels"
    """

    def __init__(self, root, transform=None, meta_classes=True):
        """
        root : chemin du dossier contenant les PNG + LG
        transform : transform PyTorch (augmentations, ToTensor, Resize…)
        meta_classes : si True, map chaque label vers une méta-classe
        """
        self.root = root
        self.transform = transform
        self.meta_classes = meta_classes

        # liste des fichiers PNG / LG
        self.images = [f for f in os.listdir(root) if f.endswith(".png")]
        self.images.sort()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.root, img_name)

        lg_name = img_name.replace(".png", ".lg")
        lg_path = os.path.join(self.root, lg_name)

        image = Image.open(img_path).convert("RGB")

        boxes = []
        labels = []

        with open(lg_path, "r", encoding='utf-8', errors='ignore') as f:
            for line in f:
                parts = [p.strip() for p in line.strip().split(",") if p.strip() != ""]
                if len(parts) < 6:
                    # fallback to whitespace splitting if commas are not reliable
                    parts = [p.strip() for p in line.strip().split() if p.strip() != ""]

                if len(parts) < 6:
                    continue

                label = parts[1]
                try:
                    xmin_s, ymin_s, xmax_s, ymax_s = parts[-4:]
                    xmin = float(xmin_s)
                    ymin = float(ymin_s)
                    xmax = float(xmax_s)
                    ymax = float(ymax_s)
                except Exception:
                    continue

                if xmax <= xmin or ymax <= ymin:
                    warnings.warn(
                            f"Found invalid bbox in '{lg_path}': [xmin={xmin}, ymin={ymin}, xmax={xmax}, ymax={ymax}]. These boxes will be skipped.")
                    continue

                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(self.map_label(label))

        # Convert to tensors; ensure correct shapes even when empty
        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}

        if self.transform:
            image = self.transform(image)

        return image, target

    def map_label(self, label):
        raw = label.split("_")[0].strip()
        if raw.isalpha():
            return 0

        if raw.isdigit():
            return 1

        if raw in {"+", "-", "=", "/", "*", "×", "÷", "^"}:
            return 2
        return 3

    def raw_label_to_id(self, raw):
        if not hasattr(self, "raw_vocab"):
            self.raw_vocab = {}
        if raw not in self.raw_vocab:
            self.raw_vocab[raw] = len(self.raw_vocab)
        return self.raw_vocab[raw]

    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])

In [ ]:
root = '../datas/FullExpressions/CROHME2019_train_png/'

In [ ]:
dataset = CROHMEDataset(
    root=root,
    transform=transforms.ToTensor(),
    meta_classes=True
)

image, target = dataset[0]
print("Image : ", image.size())
print("Target : ", target)

dataset_len = len(dataset)
train_len = int(0.8 * dataset_len)
val_len = int(0.1 * dataset_len)
test_len = dataset_len - train_len - val_len

train, val, test = torch.utils.data.random_split(
    dataset, [train_len, val_len, test_len], generator=torch.Generator().manual_seed(42)
)
print(f"Dataset sizes -> total: {dataset_len}, train: {train_len}, val: {val_len}, test: {test_len}")

Image :  torch.Size([3, 119, 500])
Target :  {'boxes': tensor([[ 10.,  40.,  39., 108.],
        [270.,  46., 320.,  72.],
        [340.,  47., 371.,  71.],
        [467.,  10., 489.,  38.],
        [121.,  38., 166.,  63.],
        [226.,  21., 265.,  65.],
        [399.,  10., 452.,  76.]]), 'labels': tensor([0, 0, 2, 1, 2, 0, 0])}
Dataset sizes -> total: 9993, train: 7994, val: 999, test: 1000


## Functions for visualization and evaluation

In [1]:
def load_image(image_path):
    """Load an image from file."""
    image = Image.open(image_path).convert("RGB")
    return image

def prepare_image(image, transform=None):
    """Prepare the image for model input."""
    if transform:
        image = transform(image)
    return image.unsqueeze(0)  # Add batch dimension

def visualize_predictions(image, boxes, labels, scores, threshold=0.4):
    """Visualize the bounding boxes and labels on the image."""
    plt.figure(figsize=(12, 8))
    plt.imshow(image.permute(1, 2, 0).numpy())

    # Filter out boxes and labels below the threshold
    for box, label, score in zip(boxes, labels, scores):
        if score >= threshold:
            x_min, y_min, x_max, y_max = box
            plt.gca().add_patch(plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                                fill=False, edgecolor='red', linewidth=3))
            plt.text(x_min, y_min, f'{label.item()}: {score:.2f}', fontsize=12, color='red')

    plt.axis('off')
    plt.show()

In [2]:
# IoU et mAP qui viennent d'Object_Segmentation

def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    Calculates intersection over union

    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct Labels of Boxes (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)

    Returns:
        tensor: Intersection over union for all examples
    """

    # Slicing idx:idx+1 in order to keep tensor dimensionality
    # Doing ... in indexing if there would be additional dimensions
    # Like for Yolo algorithm which would have (N, S, S, 4) in shape
    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    elif box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # Need clamp(0) in case they do not intersect, then we want intersection to be 0
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)
    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    return intersection / (box1_area + box2_area - intersection + 1e-6)

def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5, box_format="corners", num_classes=20
):
    """
    Calculates mean average precision

    Parameters:
        pred_boxes (list): list of lists containing all bboxes with each bboxes
        specified as [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
        true_boxes (list): Similar as pred_boxes except all the correct ones
        iou_threshold (float): threshold where predicted bboxes is correct
        box_format (str): "midpoint" or "corners" used to specify bboxes
        num_classes (int): number of classes

    Returns:
        float: mAP value across all classes given a specific IoU threshold
    """

    # list storing all AP for respective classes
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    for c in range(num_classes):
        detections = []
        ground_truths = []

        # Go through all predictions and targets,
        # and only add the ones that belong to the
        # current class c
        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        # find the amount of bboxes for each training example
        # Counter here finds how many ground truth bboxes we get
        # for each training example, so let's say img 0 has 3,
        # img 1 has 5 then we will obtain a dictionary with:
        # amount_bboxes = {0:3, 1:5}
        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        # We then go through each key, val in this dictionary
        # and convert to the following (w.r.t same example):
        # ammount_bboxes = {0:torch.tensor[0,0,0], 1:torch.tensor[0,0,0,0,0]}
        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # sort by box probabilities which is index 2
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)

        # If none exists for this class then we can safely skip
        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Only take out the ground_truths that have the same
            # training idx as detection
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            num_gts = len(ground_truth_img)
            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou = intersection_over_union(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                    box_format=box_format,
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / len(average_precisions)

## Training loop

In [13]:
# Hyperparameters
num_epochs = 3
learning_rate =0.0008
batch_size = 3
# Keep val_size if you want an absolute val count fallback, but we'll use dynamic splits
val_size = 10

val_err_array = np.array([])
train_err_array = np.array([])
nb_sample_array = np.array([])
train_loss_classifier_array = np.array([])
train_loss_objectness_array = np.array([])

# Early stopping parameters
patience =5
epochs_without_improvement = 0

# Use the Subset objects created earlier by random_split: `train`, `val`, `test`.
# If `train` or `val` don't exist yet (cell not executed), compute splits here as a fallback.
try:
    train_subset = train
    val_subset = val
except NameError:
    dataset_len = len(dataset)
    train_len = int(0.8 * dataset_len)
    val_len = int(0.1 * dataset_len)
    test_len = dataset_len - train_len - val_len
    train_subset, val_subset, _ = torch.utils.data.random_split(dataset, [train_len, val_len, test_len], generator=torch.Generator().manual_seed(42))

# Create DataLoaders for training and validation
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Load a pretrained Faster R-CNN model
#model = detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
#model = detection.ssd300_vgg16(weights="DEFAULT")
model = detection.fasterrcnn_mobilenet_v3_large_fpn(weights="DEFAULT")

# Set the requires_grad attribute of all the backbone parameters to False
for param in model.backbone.parameters():
    param.requires_grad = False
print("Backbone frozen. Only the RPN and heads will be trained.")

# Modify the model for the number of classes
num_classes = 5  # 20 classes + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Move the model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Set up the optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.SGD(params, lr=learning_rate, momentum=0.9, weight_decay=0.0005)

# Function for validation
def validate(model, val_loader):
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in val_loader:
            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            val_loss += losses.item()

    return val_loss / len(val_loader)

# Training loop
best_val_loss = float('inf')  # Initialize best validation loss
for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_loss_classifier = 0.0
    epoch_loss_objectness = 0.0
    model.train()  # Set the model to training mode
    nb_used_sample = 0 # Initialize the number of samples used in this epoch

    for images, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        # Move images and targets to the device (GPU or CPU)
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        loss_dict = model(images, targets)

        # Compute total loss
        losses = sum(loss for loss in loss_dict.values())

        # Backward pass
        losses.backward()

        # Update the weights
        optimizer.step()

        # Accumulate loss
        epoch_loss += losses.item()
        # Use .get to avoid KeyError if a particular loss term is missing
        epoch_loss_classifier += loss_dict.get('loss_classifier', torch.tensor(0.0)).item()
        epoch_loss_objectness += loss_dict.get('loss_objectness', torch.tensor(0.0)).item()
        nb_used_sample += len(images)


################ FOR VGG16 ###############
    #     # Accumulate loss - Use keys appropriate for SSD
    #     epoch_loss += losses.item()
    #     # Assuming loss_dict for SSD contains 'classification' and 'bbox_regression'
    #     if 'classification' in loss_dict:
    #         epoch_loss_classifier += loss_dict['classification'].item()
    #     if 'bbox_regression' in loss_dict:
    #          epoch_loss_objectness += loss_dict['bbox_regression'].item() # Using objectness for regression loss here
    #     nb_used_sample += batch_size


    # # Calculate average training loss for the epoch
    # train_err = epoch_loss / len(train_loader)
    # # Calculate average for classifier and regression losses only if they were accumulated
    # train_loss_classifier = epoch_loss_classifier / len(train_loader) if 'classification' in loss_dict else 0
    # train_loss_objectness = epoch_loss_objectness / len(train_loader) if 'bbox_regression' in loss_dict else 0
###########################################


    # Calculate average training loss for the epoch
    train_err = epoch_loss / len(train_loader)
    train_loss_classifier = epoch_loss_classifier / len(train_loader) if len(train_loader) > 0 else 0.0
    train_loss_objectness = epoch_loss_objectness / len(train_loader) if len(train_loader) > 0 else 0.0

    # Print epoch loss
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {train_err:.4f}, Classifier Loss: {train_loss_classifier:.4f}, Objectness Loss: {train_loss_objectness:.4f}")

    # Validate after each epoch
    val_loss = validate(model, val_loader)
    print(f"Validation Loss: {val_loss:.4f}")
    train_err_array = np.append(train_err_array, train_err)
    val_err_array = np.append(val_err_array, val_loss)
    nb_sample_array = np.append(nb_sample_array, nb_used_sample)
    train_loss_classifier_array = np.append(train_loss_classifier_array, train_loss_classifier)
    train_loss_objectness_array = np.append(train_loss_objectness_array, train_loss_objectness)

    # Save the model weights if validation loss has improved
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'faster_rcnn_voc_best.pth')
        print(f"Model weights saved. New best validation loss: {best_val_loss:.4f}")
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= patience:
        print(f"Early stopping after {patience} epochs without improvement.")
        break

# Final message
print("Training complete.")

Backbone frozen. Only the RPN and heads will be trained.


Epoch 1/3:   3%|▎         | 75/2665 [00:52<29:57,  1.44it/s]



KeyboardInterrupt: 

In [ ]:
# IoU et mAP qui viennent d'Object_Segmentation

def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    """
    Calculates intersection over union

    Parameters:
        boxes_preds (tensor): Predictions of Bounding Boxes (BATCH_SIZE, 4)
        boxes_labels (tensor): Correct Labels of Boxes (BATCH_SIZE, 4)
        box_format (str): midpoint/corners, if boxes (x,y,w,h) or (x1,y1,x2,y2)

    Returns:
        tensor: Intersection over union for all examples
    """

    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    elif box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = torch.max(box1_x1, box2_x1)
    y1 = torch.max(box1_y1, box2_y1)
    x2 = torch.min(box1_x2, box2_x2)
    y2 = torch.min(box1_y2, box2_y2)

    # Need clamp(0) in case they do not intersect, then we want intersection to be 0
    intersection = (x2 - x1).clamp(min=0) * (y2 - y1).clamp(min=0)
    box1_area = (box1_x2 - box1_x1).abs() * (box1_y2 - box1_y1).abs()
    box2_area = (box2_x2 - box2_x1).abs() * (box2_y2 - box2_y1).abs()

    return intersection / (box1_area + box2_area - intersection + 1e-6)


def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5, box_format="corners", num_classes=20
):
    """
    Calculates mean average precision

    Parameters:
        pred_boxes (list): list of lists containing all bboxes with each bboxes
        specified as [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
        true_boxes (list): Similar as pred_boxes except all the correct ones
        iou_threshold (float): threshold where predicted bboxes is correct
        box_format (str): "midpoint" or "corners" used to specify bboxes
        num_classes (int): number of classes

    Returns:
        float: mAP value across all classes given a specific IoU threshold
    """

    average_precisions = []
    epsilon = 1e-6

    for c in range(num_classes):
        detections = []
        ground_truths = []

        # Collect detections and ground truths for class c
        for detection in pred_boxes:
            # detection[1] may be int or tensor-like
            if int(detection[1]) == c:
                detections.append(detection)

        for true_box in true_boxes:
            if int(true_box[1]) == c:
                ground_truths.append(true_box)

        amount_bboxes = Counter([gt[0] for gt in ground_truths])

        for key, val in amount_bboxes.items():
            amount_bboxes[key] = torch.zeros(val)

        # sort by confidence score (index 2)
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_bboxes = len(ground_truths)

        if total_true_bboxes == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            # Ground truths for the same image index
            ground_truth_img = [bbox for bbox in ground_truths if bbox[0] == detection[0]]

            best_iou = 0.0
            best_gt_idx = -1

            for idx, gt in enumerate(ground_truth_img):
                # Compute IoU and convert to float for safe comparisons
                iou_tensor = intersection_over_union(
                    torch.tensor(detection[3:], dtype=torch.float32),
                    torch.tensor(gt[3:], dtype=torch.float32),
                    box_format=box_format,
                )
                iou = float(iou_tensor.item()) if torch.is_tensor(iou_tensor) else float(iou_tensor)

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth once
                if amount_bboxes[detection[0]][best_gt_idx] == 0:
                    TP[detection_idx] = 1
                    amount_bboxes[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)

        # Ensure tensors are float and prepend start points for integration
        precisions = torch.cat((torch.tensor([1.0]), precisions))
        recalls = torch.cat((torch.tensor([0.0]), recalls))

        # Numerical integration (area under precision-recall curve)
        ap = float(torch.trapz(precisions, recalls).item())
        average_precisions.append(ap)

    if len(average_precisions) == 0:
        return 0.0

    return sum(average_precisions) / len(average_precisions)


In [17]:
# Secondary installation attempt kept robust: use python -m pip to avoid broken pip binary
import sys, subprocess
try:
    import pix2text
except Exception:
    print('pix2text not found — installing via python -m pip')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pix2text>=1.1'])
    import pix2text
print('pix2text is available')

pix2text not found — installing via python -m pip
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Installing build dependencies: started
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Getting requirements to build wheel: f

In [21]:
from pix2text import Pix2Text

# Initialize Pix2Text: try GPU first, fall back to CPU if ONNX Runtime doesn't expose CUDAExecutionProvider
try:
    p2t = Pix2Text.from_config(device='cuda')  # attempt GPU/ONNXRuntime CUDAExecutionProvider
    print('Pix2Text initialized on CUDA device')
except Exception as e:
    # Common failure: onnxruntime not built with GPU support -> ValueError about CUDAExecutionProvider
    print('GPU initialization failed (will fall back to CPU):', e)
    print('Initializing Pix2Text on CPU...')
    p2t = Pix2Text.from_config(device='cpu')
    print('Pix2Text initialized on CPU')

# Example inference (update path to an actual image in your workspace)
img_path = '../datas/FullExpressions/CROHME2019_train_png/001-equation000.png'
try:
    res = p2t.recognize_formula(img_path, return_text=True)
    print(res)
except Exception as e:
    print('Error running recognition:', e)

[INFO] 2025-12-02 15:21:36,471 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-02 15:21:36,473 [RapidOCR] main.py:53: Using /home/raclax/.cnstd/1.2/ppocr/ch_PP-OCRv5_det/ch_PP-OCRv5_det_infer.onnx
[WARNING] 2025-12-02 15:21:36,475 [RapidOCR] provider_config.py:93: CUDAExecutionProvider is not in available providers (['AzureExecutionProvider', 'CPUExecutionProvider']). Use AzureExecutionProvider inference by default.
[INFO] 2025-12-02 15:21:36,476 [RapidOCR] provider_config.py:168: If you want to use CUDAExecutionProvider acceleration, you must do:(For reference only) If you want to use GPU acceleration, you must do:
[INFO] 2025-12-02 15:21:36,477 [RapidOCR] provider_config.py:168: First, uninstall all onnxruntime packages in current environment.
[INFO] 2025-12-02 15:21:36,478 [RapidOCR] provider_config.py:168: Second, install onnxruntime-gpu by `pip install onnxruntime-gpu`.
[INFO] 2025-12-02 15:21:36,479 [RapidOCR] provider_config.py:168: Note the onnxruntime-gpu 

GPU initialization failed (will fall back to CPU): Asked to use CUDAExecutionProvider as an ONNX Runtime execution provider, but the available execution providers are ['AzureExecutionProvider', 'CPUExecutionProvider'].
Initializing Pix2Text on CPU...


[INFO] 2025-12-02 15:21:37,737 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2025-12-02 15:21:37,739 [RapidOCR] main.py:53: Using /home/raclax/.cnstd/1.2/ppocr/ch_PP-OCRv5_det/ch_PP-OCRv5_det_infer.onnx
[INFO] 2025-12-02 15:21:37,739 [RapidOCR] main.py:53: Using /home/raclax/.cnstd/1.2/ppocr/ch_PP-OCRv5_det/ch_PP-OCRv5_det_infer.onnx
Could not find any ONNX files with standard file name decoder_model_merged.onnx, files found: [PosixPath('decoder_model.onnx'), PosixPath('encoder_model.onnx')]. Please make sure to pass a `file_name` and/or `subfolder` argument to `from_pretrained` when loading an ONNX file with non-standard file names.
Could not find any ONNX files with standard file name decoder_model_merged.onnx, files found: [PosixPath('decoder_model.onnx'), PosixPath('encoder_model.onnx')]. Please make sure to pass a `file_name` and/or `subfolder` argument to `from_pretrained` when loading an ONNX file with non-standard file names.


Pix2Text initialized on CPU


100%|██████████| 1/1 [00:00<00:00,  1.64it/s]

y=A x+A^{2}


In [ ]:
from ultralytics import YOLO

# Charger un modèle pré-entraîné
model = YOLO('yolov8s.pt')

# Entraîner
results = model.train(
    data='YOLO_dataset/data.yaml',
    epochs=10,
    imgsz=256,
    batch=16,
    name='math_symbols_detector'
)

# Tester
results = model.val()

In [10]:
results = model.predict('YOLO_dataset/images/train/001-equation002.png', save=True, conf=0.5)

result = results[0]
boxes = result.boxes
print(boxes)


image 1/1 /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/YOLO_dataset/images/train/001-equation002.png: 64x256 4 letters, 1 other, 12.6ms
Speed: 11.0ms preprocess, 12.6ms inference, 2.0ms postprocess per image at shape (1, 3, 64, 256)
Results saved to /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/runs/detect/predict2
image 1/1 /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/YOLO_dataset/images/train/001-equation002.png: 64x256 4 letters, 1 other, 12.6ms
Speed: 11.0ms preprocess, 12.6ms inference, 2.0ms postprocess per image at shape (1, 3, 64, 256)
Results saved to /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/runs/detect/predict2
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0., 3., 0., 0., 0.], device='cuda:0')
conf: tensor([0.6446, 0.6234, 0.6034, 0.5456, 0.5264], device='cuda:0')
data: tensor([[ 38.5873,  31.3881,  64.6572,  45.0779,   0.6446,   0.0000],
        [434.3517,  21

In [ ]:
#for images on val set compute the mean iou using the ground truth bounding box coordinates from the labels folder 
val_labels_path = 'YOLO_dataset/labels/val/'
val_images_path = 'YOLO_dataset/images/val/'
ious = []
for img_file in os.listdir(val_images_path):
    results = model.predict (img_file, save =True, conf=0.5)
    result = results[0]
    boxes = result.boxes

In [ ]:
results = model.predict('YOLO_dataset/images/train/001-equation002.png', save=True, conf=0.5)
result = results[0]

boxes = result.boxes
print("Boxes (xyxy):", boxes.xyxy)
print("Boxes (xywhn):", boxes.xywhn)
print("Confidence:", boxes.conf)
print("Classes:", boxes.cls)

for i, box in enumerate(boxes):
    xyxy = box.xyxy[0].tolist()
    conf = box.conf[0].item() 
    cls_id = int(box.cls[0].item()) 
    cls_name = result.names[cls_id] 
    
    print(f"Box {i}: {cls_name} (conf: {conf:.2f}) -> [{xyxy[0]:.1f}, {xyxy[1]:.1f}, {xyxy[2]:.1f}, {xyxy[3]:.1f}]")


image 1/1 /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/YOLO_dataset/images/train/001-equation002.png: 64x256 4 letters, 1 other, 13.9ms
Speed: 10.7ms preprocess, 13.9ms inference, 2.1ms postprocess per image at shape (1, 3, 64, 256)
Results saved to /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/runs/detect/predict2
image 1/1 /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/YOLO_dataset/images/train/001-equation002.png: 64x256 4 letters, 1 other, 13.9ms
Speed: 10.7ms preprocess, 13.9ms inference, 2.1ms postprocess per image at shape (1, 3, 64, 256)
Results saved to /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/runs/detect/predict2
Boxes (xyxy): tensor([[ 38.5873,  31.3881,  64.6572,  45.0779],
        [434.3517,  21.3157, 458.3569,  38.4283],
        [474.6455,  26.6818, 489.8564,  59.9949],
        [203.8767,   9.0561, 222.0511,  44.6062],
        [147.3532,  15.3053, 170.3515,  47.7883]], device='cuda:0

In [14]:
# Cellule d'évaluation : compare predictions YOLO (Ultralytics) aux labels YOLO (fichiers .txt)
from pathlib import Path
from PIL import Image
import numpy as np
import os

def yolo_line_to_xyxy(line, img_w, img_h):
    parts = line.strip().split()
    if len(parts) < 5:
        raise ValueError(f'Malformed label line: {line!r}')
    cls = int(float(parts[0]))
    xc = float(parts[1]) * img_w
    yc = float(parts[2]) * img_h
    w = float(parts[3]) * img_w
    h = float(parts[4]) * img_h
    xmin = xc - w/2
    ymin = yc - h/2
    xmax = xc + w/2
    ymax = yc + h/2
    return [xmin, ymin, xmax, ymax], cls

def iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = a
    bx1, by1, bx2, by2 = b
    ix1 = max(ax1, bx1)
    iy1 = max(ay1, by1)
    ix2 = min(ax2, bx2)
    iy2 = min(ay2, by2)
    iw = max(0.0, ix2 - ix1)
    ih = max(0.0, iy2 - iy1)
    inter = iw * ih
    area_a = max(0.0, ax2-ax1) * max(0.0, ay2-ay1)
    area_b = max(0.0, bx2-bx1) * max(0.0, by2-by1)
    union = area_a + area_b - inter
    return inter/union if union>0 else 0.0

def evaluate_yolo_labels(images_dir, labels_dir, model, iou_threshold=0.5, conf=0.25, max_images=None):
    images_dir = Path(images_dir)
    labels_dir = Path(labels_dir)
    imgs = sorted([p for p in images_dir.iterdir() if p.suffix.lower() in {'.png','.jpg','.jpeg'}])
    if max_images:
        imgs = imgs[:max_images]

    total_TP = total_FP = total_FN = 0
    iou_matches = []

    for img_path in imgs:
        with Image.open(img_path) as im:
            W,H = im.size

        gt_path = labels_dir / (img_path.stem + '.txt')
        gt_boxes = []
        gt_classes = []
        if gt_path.exists():
            for line in gt_path.read_text(encoding='utf-8').splitlines():
                if not line.strip():
                    continue
                try:
                    box, cls = yolo_line_to_xyxy(line, W, H)
                except Exception as e:
                    print(f'Skipping malformed label in {gt_path}: {e}')
                    continue
                gt_boxes.append(box)
                gt_classes.append(cls)

        # predictions (Ultralytics model.predict)
        reslist = model.predict(str(img_path), conf=conf)
        res = reslist[0]
        if hasattr(res, 'boxes') and len(res.boxes) > 0:
            try:
                pred_boxes = res.boxes.xyxy.cpu().numpy()
                pred_cls = res.boxes.cls.cpu().numpy().astype(int)
            except Exception:
                pred_boxes = np.array(res.boxes.xyxy)
                pred_cls = np.array([int(x) for x in res.boxes.cls])
        else:
            pred_boxes = np.zeros((0,4))
            pred_cls = np.array([])

        used = set()
        matched = 0
        for gi, (gbox, gcls) in enumerate(zip(gt_boxes, gt_classes)):
            best_iou = 0.0
            best_pi = -1
            for pi, (pbox, pcl) in enumerate(zip(pred_boxes, pred_cls)):
                if pi in used:
                    continue
                if int(pcl) != int(gcls):
                    continue
                cur_iou = iou_xyxy(gbox, pbox)
                if cur_iou > best_iou:
                    best_iou = cur_iou
                    best_pi = pi
            if best_pi >=0 and best_iou >= iou_threshold:
                matched += 1
                used.add(best_pi)
                iou_matches.append(best_iou)
        TP = matched
        FP = len(pred_boxes) - matched
        FN = len(gt_boxes) - matched
        total_TP += TP
        total_FP += FP
        total_FN += FN
        print(f'{img_path.name}: GT={len(gt_boxes)} Pred={len(pred_boxes)} TP={TP} FP={FP} FN={FN} meanIoU_matched={(np.mean(iou_matches[-matched:]) if matched>0 else 0.0):.3f}')

    precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP)>0 else 0.0
    recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN)>0 else 0.0
    mean_iou = float(np.mean(iou_matches)) if len(iou_matches)>0 else 0.0
    print('=== SUMMARY ===')
    print(f'Total GT boxes: {total_TP + total_FN}')
    print(f'Total Pred boxes: {total_TP + total_FP}')
    print(f'Total TP: {total_TP}, FP: {total_FP}, FN: {total_FN}')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, Mean IoU (matched): {mean_iou:.4f}')
    return {'total_TP': total_TP, 'total_FP': total_FP, 'total_FN': total_FN, 'precision': precision, 'recall': recall, 'mean_iou': mean_iou}

# Exemple d'appel - mettez à jour les chemins si nécessaire :
metrics = evaluate_yolo_labels('YOLO_dataset/images/val/', 'YOLO_dataset/labels/val/', model, iou_threshold=0.5, conf=0.25)


image 1/1 /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/YOLO_dataset/images/val/001-equation000.png: 64x256 3 letters, 1 digit, 2 operators, 1 other, 13.6ms
Speed: 11.1ms preprocess, 13.6ms inference, 2.7ms postprocess per image at shape (1, 3, 64, 256)
001-equation000.png: GT=7 Pred=7 TP=6 FP=1 FN=1 meanIoU_matched=0.941

image 1/1 /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/YOLO_dataset/images/val/001-equation000.png: 64x256 3 letters, 1 digit, 2 operators, 1 other, 13.6ms
Speed: 11.1ms preprocess, 13.6ms inference, 2.7ms postprocess per image at shape (1, 3, 64, 256)
001-equation000.png: GT=7 Pred=7 TP=6 FP=1 FN=1 meanIoU_matched=0.941

image 1/1 /home/raclax/Documents/M2/Part2/DL2/Project/DL_project_marie_clara/YOLO_dataset/images/val/001-equation0011.png: 64x256 2 letters, 4 digits, 3 operators, 5 others, 13.7ms
Speed: 1.0ms preprocess, 13.7ms inference, 2.6ms postprocess per image at shape (1, 3, 64, 256)
001-equation0011.png: GT=12 Pr

# RTDETR

In [ ]:
from ultralytics import RTDETR

model = RTDETR('rtdetr-l.pt')

results = model.train(
    data='YOLO_dataset/data.yaml',
    epochs=5,
    imgsz=640,
    batch=4,
    name='math_rtdetr'
)

metrics = model.val()

predictions = model.predict('YOLO_dataset/images/001-equation001.png', conf=0.5)

In [ ]:
results = model.predict('YOLO_dataset/images/train/001-equation002.png', save=True, conf=0.5)
# Accéder au premier résultat
result = results[0]

# ===== BOUNDING BOXES =====
boxes = result.boxes

# Coordonnées des boxes (format xyxy : xmin, ymin, xmax, ymax)
print("Boxes (xyxy):", boxes.xyxy)

# Coordonnées normalisées (format xywhn : x_center, y_center, width, height normalisés)
print("Boxes (xywhn):", boxes.xywhn)

# Confiance de chaque prédiction
print("Confidence:", boxes.conf)

# Classes prédites (indices)
print("Classes:", boxes.cls)

# ===== DÉTAILS COMPLETS =====
for i, box in enumerate(boxes):
    xyxy = box.xyxy[0].tolist()  # [xmin, ymin, xmax, ymax]
    conf = box.conf[0].item() 
    cls_id = int(box.cls[0].item()) 
    cls_name = result.names[cls_id] 
    
    print(f"Box {i}: {cls_name} (conf: {conf:.2f}) -> [{xyxy[0]:.1f}, {xyxy[1]:.1f}, {xyxy[2]:.1f}, {xyxy[3]:.1f}]")